## Trabajo con features elegidos a mano, cuando esté lo de feature selection los modificamos

In [132]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime

In [133]:
keywords_numerical = pd.read_csv('features/train/features_keywords_numerical.csv').drop(columns=['Unnamed: 0'])
text_numerical = pd.read_csv('features/train/features_text_numerical.csv').drop(columns=['Unnamed: 0','target'])
location_in_text_numerical = pd.read_csv('features/train/features_location_in_text_numerical.csv').drop(columns=['target','Unnamed: 0'])

In [134]:
train_features = location_in_text_numerical.merge(text_numerical, on='id')
train_features = train_features.merge(keywords_numerical, on='id')
train_features.head()

,id,has_country,has_city,has_county,has_state,has_continent,#palabras,#palabras_unicas,#caracteres,#stopwords,...,#mayusculas_binned,#silabas_binned,#caracteres_especiales_binned,target,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency
0,1,0,0,0,0,0,13,13,71,5,...,0,1,0,1,1,1,1,0.004525,1,0.227027
1,4,1,0,0,0,1,7,7,40,0,...,0,0,0,1,1,1,1,0.004525,0,0.237838
2,5,0,0,0,0,0,22,20,135,9,...,0,3,1,1,1,1,1,0.004525,0,0.210811
3,6,1,0,0,1,1,8,8,67,1,...,0,1,0,1,1,1,1,0.004525,0,0.210811
4,7,1,0,0,1,1,16,15,90,6,...,0,2,0,1,1,1,1,0.004525,0,0.237838


In [135]:
keywords_numerical = pd.read_csv('features/test/features_keywords_numerical.csv').drop(columns=['Unnamed: 0'])
text_numerical = pd.read_csv('features/test/features_text_numerical.csv').drop(columns=['Unnamed: 0'])
location_in_text_numerical = pd.read_csv('features/test/features_location_in_text_numerical.csv').drop(columns=['Unnamed: 0'])

In [136]:
test_features = location_in_text_numerical.merge(text_numerical, on='id')
test_features = test_features.merge(keywords_numerical, on='id').fillna(0)
test_features.head()

,id,has_country,has_city,has_county,has_state,has_continent,#palabras,#palabras_unicas,#caracteres,#stopwords,...,#capitalize_binned,#mayusculas_binned,#silabas_binned,#caracteres_especiales_binned,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency
0,0,0,0,0,0,0,6,6,36,1,...,0,0,0,0,1,1,1,0.004525,0,0.183784
1,2,0,0,0,0,0,9,9,66,2,...,0,0,1,1,1,1,1,0.004525,1,0.075676
2,3,0,0,0,0,0,19,19,98,8,...,0,0,1,0,1,1,1,0.004525,0,0.075676
3,9,0,0,0,0,0,4,4,42,0,...,1,0,0,1,1,1,1,0.004525,0,0.102703
4,11,1,0,0,0,1,8,8,47,2,...,1,0,1,0,1,1,1,0.004525,0,0.070270


#####################################

In [137]:
def save_prediction_accuracy(modelo,x_train,y_train,x_test,y_test):
    with open("prediction_history.csv", "a") as myfile:
        predicted = modelo.predict(x_train)
        train_prediction = str(metrics.accuracy_score(y_train,predicted))
        print('Score para x_train: '+ train_prediction)
        predicted = modelo.predict(x_test)
        test_prediction = str(metrics.accuracy_score(y_test,predicted))
        print('Score para x_test: '+ test_prediction)
        params = str(modelo)
        print('Hiperparametros: '+ str(modelo))
        myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

In [188]:
def save_prediction(model, test, name):
    df_prediccion = test[['id']].copy()
    predicted = model.predict(test.drop(columns='id'))
    df_prediccion['target'] = predicted
    df_prediccion.to_csv(name+'.csv', index=None)
    return

In [139]:
train_df = train_features.drop(columns = ['target'])
train_df = train_df.fillna(0)

#Split the CountVector vectorized data into train and test datasets for model training and testing
X_train, X_test, y_train, y_test =train_test_split(train_df,train_features.target,test_size=0.2,random_state=2020)

## Estimación de accuracy

aca hay mas cosas interesantes https://www.ritchieng.com/machine-learning-evaluate-classification-model/

In [140]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [170]:
def fit_and_predict(model,X_train,y_train,X_test,y_test):    
    clf = model
    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    confusion_matrix(y_test,predictions)
    conf = metrics.confusion_matrix(y_test, predictions)
    conf = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    print_cm(conf, ['true','false'])
    print('-'*50)
    print(classification_report(y_test,predictions))
    print('-'*50)
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(clf.score(X_train, y_train)*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test,predictions)*100)))
    print('*'*50)

In [142]:
models=[LogisticRegression(C=1.0),SVC(),MultinomialNB(),DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=5),RandomForestClassifier(),GaussianNB(),SGDClassifier(loss="hinge", penalty="l2", max_iter=5)]

In [171]:
for model in models:
    fit_and_predict(model,X_train.drop(columns='id'), y_train,X_test.drop(columns='id'),y_test)

/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


           true false 
     true   0.9   0.1 
    false   0.5   0.5 
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.67      0.85      0.75       849
           1       0.72      0.48      0.57       674

    accuracy                           0.68      1523
   macro avg       0.69      0.66      0.66      1523
weighted avg       0.69      0.68      0.67      1523

--------------------------------------------------
LogisticRegression()
--------------------------------------------------
Accuracy of classifier on training set:70%
--------------------------------------------------
Accuracy of classifier on test set:68%
**************************************************
           true false 
     true   0.8   0.2 
    false   0.6   0.4 
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.63      0.83      0.72       849
           1     

/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


## Estimación de accuracy con Cross-Validation ?

In [144]:
def fit_and_predict_cross_val(model,X_train,y_train,X_test,y_test):
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(cross_val_score(model, X_train, y_train, scoring='accuracy', cv = 10).mean()*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(cross_val_score(model, X_test, y_test, scoring='accuracy', cv = 10).mean()*100)))
    print('*'*50)

In [173]:
for model in models:
    fit_and_predict_cross_val(model,X_train.drop(columns='id'), y_train,X_test.drop(columns='id'),y_test)

LogisticRegression()
--------------------------------------------------


/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model

Accuracy of classifier on training set:69%
--------------------------------------------------


/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model

Accuracy of classifier on test set:68%
**************************************************
SVC()
--------------------------------------------------
Accuracy of classifier on training set:64%
--------------------------------------------------
Accuracy of classifier on test set:61%
**************************************************
MultinomialNB()
--------------------------------------------------
Accuracy of classifier on training set:64%
--------------------------------------------------
Accuracy of classifier on test set:63%
**************************************************
DecisionTreeClassifier()
--------------------------------------------------
Accuracy of classifier on training set:64%
--------------------------------------------------
Accuracy of classifier on test set:62%
**************************************************
KNeighborsClassifier()
--------------------------------------------------
Accuracy of classifier on training set:65%
-----------------------------------------

/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/hom

Accuracy of classifier on training set:55%
--------------------------------------------------
Accuracy of classifier on test set:55%
**************************************************


/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/home/daniela/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/hom

## Random Forest

### Random Search

In [172]:
from sklearn.model_selection import RandomizedSearchCV # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [174]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42)# Fit the random search model
rf_random.fit(X_train.drop(columns='id'), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   1.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   1.5s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   1.5s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, total=   5.6s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, total=   5.5s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600,

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [175]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [176]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    accuracy = round(accuracy_score(test_labels,predictions)*100)
    print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train.drop(columns='id'), y_train)
base_accuracy = evaluate(base_model, X_test.drop(columns='id'), y_test)

Model Performance
Accuracy = 71.00%.


In [177]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test.drop(columns='id'), y_test)

Model Performance
Accuracy = 71.00%.


In [178]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 0.00%.


In [179]:
best_random

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=200)

In [180]:
save_prediction_accuracy(best_random, X_train.drop(columns='id'), y_train, X_test.drop(columns='id'), y_test)

Score para x_train: 0.7932676518883416
Score para x_test: 0.706500328299409
Hiperparametros: RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=200)


## LIGHT GBM

In [201]:
# Create a training and testing dataset
train_set = lgb.Dataset(data = X_train.drop(columns='id'), label = y_train)
test_set = lgb.Dataset(data = X_test.drop(columns='id'), label = y_test)

In [202]:
N_FOLDS = 5
MAX_EVALS = 5

# Get default hyperparameters
model = lgb.LGBMClassifier()
default_params = model.get_params()

# Remove the number of estimators because we set this to 10000 in the cv call
del default_params['n_estimators']

# Cross validation with early stopping
cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, 
                    metrics = 'auc', nfold = N_FOLDS, seed = 42)

/home/daniela/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


In [203]:
print('The maximum validation ROC AUC was: {:.5f} with a standard deviation of {:.5f}.'.format(cv_results['auc-mean'][-1], cv_results['auc-stdv'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

The maximum validation ROC AUC was: 0.77335 with a standard deviation of 0.00974.
The optimal number of boosting rounds (estimators) was 57.


In [204]:
from sklearn.metrics import roc_auc_score

In [205]:
# Optimal number of esimators found in cv
model.n_estimators = len(cv_results['auc-mean'])

# Train and make predicions with model
model.fit(X_train.drop(columns='id'), y_train)
preds = model.predict_proba(X_test.drop(columns='id'))[:, 1]
baseline_auc = roc_auc_score(y_test, preds)

print('The baseline model scores {:.5f} ROC AUC on the test set.'.format(baseline_auc))

The baseline model scores 0.76969 ROC AUC on the test set.


In [206]:
save_prediction_accuracy(model, X_train.drop(columns='id'), y_train, X_test.drop(columns='id'), y_test)

Score para x_train: 0.7975369458128079
Score para x_test: 0.7117531188443861
Hiperparametros: LGBMClassifier(n_estimators=57)


### TUNING [se me rompe todo cuando quiero ejecutar los search]

In [82]:
def objective(hyperparameters, iteration):
    """Objective function for grid and random search. Returns
       the cross validation score from a set of hyperparameters."""
    
    # Number of estimators will be found using early stopping
    if 'n_estimators' in hyperparameters.keys():
        del hyperparameters['n_estimators']
    
     # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, nfold = N_FOLDS, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 42)
    
    # results to retun
    score = cv_results['auc-mean'][-1]
    estimators = len(cv_results['auc-mean'])
    hyperparameters['n_estimators'] = estimators 
    
    return [score, hyperparameters, iteration]

In [29]:
# ESTO DEBERIA DAR LO MISMO QUE ANTES PORQUE LE PASO LOS PARAMETROS DEFAULT
score, params, iteration = objective(default_params, 1)

print('The cross-validation ROC AUC was {:.5f}.'.format(score))

The cross-validation ROC AUC was 0.80482.


In [40]:
# Hyperparameter grid
param_grid = {
    'boosting_type': ['gbdt', 'goss'],
    'num_leaves': list(range(20, 150)),
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000)),
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10)),
    'subsample': list(np.linspace(0.5, 1, 100)),
    'is_unbalance': [True, False]
}

In [37]:
# Dataframes for random and grid search
random_results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))

grid_results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))


In [38]:
import itertools

def grid_search(param_grid, max_evals = MAX_EVALS):
    """Grid search algorithm (with limit on max evals)"""
    
    # Dataframe to store results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))
    
    # https://codereview.stackexchange.com/questions/171173/list-all-possible-permutations-from-a-python-dictionary-of-lists
    keys, values = zip(*param_grid.items())
    
    i = 0
    
    # Iterate through every possible combination of hyperparameters
    for v in itertools.product(*values):
        
        # Create a hyperparameter dictionary
        hyperparameters = dict(zip(keys, v))
        
        # Set the subsample ratio accounting for boosting type
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting_type'] == 'goss' else hyperparameters['subsample']
        
        # Evalute the hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
        
        i += 1
        
        # Normally would not limit iterations
        if i > MAX_EVALS:
            break
       
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    
    return results

In [50]:
#grid_results = grid_search(param_grid)

#print('The best validation score was {:.5f}'.format(grid_results.loc[0, 'score']))
#print('\nThe best hyperparameters were:')

#import pprint
#pprint.pprint(grid_results.loc[0, 'params'])

In [49]:
# Get the best parameters
#grid_search_params = grid_results.loc[0, 'params']

# Create, train, test model
#model = lgb.LGBMClassifier(**grid_search_params, random_state=42)
#model.fit(X_train, y_train)

#preds = model.predict_proba(X_test)[:, 1]

#print('The best model from grid search scores {:.5f} ROC AUC on the test set.'.format(roc_auc_score(y_test, preds)))

In [ ]:
################ random search

In [44]:
import random

In [45]:
def random_search(param_grid, max_evals = MAX_EVALS):
    """Random search for hyperparameter optimization"""
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(MAX_EVALS)))
    
    # Keep searching until reach max evaluations
    for i in range(MAX_EVALS):
        
        # Choose random hyperparameters
        hyperparameters = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting_type'] == 'goss' else hyperparameters['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
    
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    return results 

In [47]:
#random_results = random_search(param_grid)

#print('The best validation score was {:.5f}'.format(random_results.loc[0, 'score']))
#print('\nThe best hyperparameters were:')

#import pprint
#pprint.pprint(random_results.loc[0, 'params'])

In [168]:
#actuals_onehot = pd.get_dummies(y_train).values
#false_positive_rate, recall, thresholds = roc_curve(actuals_onehot[0], np.round(predictions)[0])
#roc_auc = auc(false_positive_rate, recall)
#print("AUC score ",roc_auc)

## PREDICCIONES

In [190]:
# RANDOM FOREST
save_prediction(best_random, test_features,'RF_1')

In [191]:
# LIGHTGBM

In [207]:
# RANDOM FOREST
save_prediction(model, test_features,'LGBM_1')